# Trading Strategy & Backtesting (Regime-Aware)
## Quantitative Trading System – NIFTY (Daily)

This notebook implements a **5/15 EMA crossover trading strategy**
filtered using **HMM-based market regimes** generated in a previous module.

### Tasks Covered
- Task 4.1: Strategy Implementation
- Task 4.2: Backtesting & Evaluation

### Strategy Logic
- Trend-following EMA crossover
- Regime-aware entries
- No trades in sideways markets (Regime 0)

### Input Files
- nifty_features_daily.csv
- nifty_regimes_daily.csv


In [9]:
import pandas as pd
import numpy as np

In [10]:
features_df = pd.read_csv("nifty_features_daily.csv")
regime_df   = pd.read_csv("nifty_regimes_daily.csv")

features_df['date'] = pd.to_datetime(features_df['date'])
regime_df['date']   = pd.to_datetime(regime_df['date'])

In [11]:
features_df = pd.read_csv("nifty_features_daily.csv")
regime_df   = pd.read_csv("nifty_regimes_daily.csv")

features_df['date'] = pd.to_datetime(features_df['date'])
regime_df['date']   = pd.to_datetime(regime_df['date'])

df = features_df.merge(regime_df, on='date', how='inner')
df = df.sort_values('date').reset_index(drop=True)

In [12]:
split = int(0.7 * len(df))

train_df = df.iloc[:split].copy()
test_df  = df.iloc[split:].copy()

# STRATEGY IMPLEMENTATION

In [13]:
df['ema_cross_up'] = (
    (df['ema_5'] > df['ema_15']) &
    (df['ema_5'].shift(1) <= df['ema_15'].shift(1))
)

df['ema_cross_down'] = (
    (df['ema_5'] < df['ema_15']) &
    (df['ema_5'].shift(1) >= df['ema_15'].shift(1))
)

In [15]:
df['position'] = 0

In [16]:
# LONG Entry
df.loc[
    (df['ema_cross_up']) &
    (df['regime'] == 1),
    'position'
] = 1

# LONG Exit
df.loc[
    (df['ema_cross_down']),
    'position'
] = 0

In [17]:
# SHORT Entry
df.loc[
    (df['ema_cross_down']) &
    (df['regime'] == -1),
    'position'
] = -1

# SHORT Exit
df.loc[
    (df['ema_cross_up']),
    'position'
] = 0

In [18]:
df['position'] = df['position'].replace(0, np.nan).ffill().fillna(0)

In [19]:
df['position'] = df['position'].replace(0, np.nan).ffill().fillna(0)

In [21]:
# Strategy returns (enter at next candle open)
df['strategy_return'] = df['position'].shift(1) * df['spot_return']

# BACKTESTING

In [22]:
test_df = df.iloc[split:].copy()
test_df['equity'] = (1 + test_df['strategy_return']).cumprod()

In [23]:
def sharpe_ratio(returns):
    return np.sqrt(252) * returns.mean() / returns.std()

def sortino_ratio(returns):
    downside = returns[returns < 0]
    return np.sqrt(252) * returns.mean() / downside.std()

def max_drawdown(equity):
    roll_max = equity.cummax()
    drawdown = equity / roll_max - 1
    return drawdown.min()m

In [24]:
total_return = test_df['equity'].iloc[-1] - 1
sharpe  = sharpe_ratio(test_df['strategy_return'])
sortino = sortino_ratio(test_df['strategy_return'])
mdd     = max_drawdown(test_df['equity'])
calmar  = total_return / abs(mdd)

win_rate = (test_df['strategy_return'] > 0).mean()

profit_factor = (
    test_df[test_df['strategy_return'] > 0]['strategy_return'].sum() /
    abs(test_df[test_df['strategy_return'] < 0]['strategy_return'].sum())
)

total_trades = (df['position'].diff().abs() > 0).sum()

In [25]:
results = pd.Series({
    "Total Return": total_return,
    "Sharpe Ratio": sharpe,
    "Sortino Ratio": sortino,
    "Calmar Ratio": calmar,
    "Max Drawdown": mdd,
    "Win Rate": win_rate,
    "Profit Factor": profit_factor,
    "Total Trades": total_trades
})

results

,0
Total Return,-0.031959
Sharpe Ratio,-1.348563
Sortino Ratio,-2.219901
Calmar Ratio,-0.479412
Max Drawdown,-0.066663
Win Rate,0.520000
Profit Factor,0.811662
Total Trades,1.000000


In [26]:
baseline_ml_df = df[[
    'date',
    'close_spot',
    'regime',
    'ema_5',
    'ema_15',
    'position',
    'spot_return',
    'strategy_return'
]].copy()

# Equity curve
baseline_ml_df['equity'] = (1 + baseline_ml_df['strategy_return']).cumprod()

# Binary ML target:
# 1 = profitable trade, 0 = not profitable
baseline_ml_df['ml_target'] = (baseline_ml_df['strategy_return'] > 0).astype(int)

# Signal strength (useful ML feature)
baseline_ml_df['ema_spread'] = baseline_ml_df['ema_5'] - baseline_ml_df['ema_15']

# Lag features (important for ML)
baseline_ml_df['ret_lag_1'] = baseline_ml_df['spot_return'].shift(1)
baseline_ml_df['ret_lag_2'] = baseline_ml_df['spot_return'].shift(2)
baseline_ml_df['ret_lag_3'] = baseline_ml_df['spot_return'].shift(3)

baseline_ml_df = baseline_ml_df.dropna().reset_index(drop=True)

baseline_ml_df.to_csv("baseline_strategy_ml_ready.csv", index=False)

## Strategy & Backtesting Summary

### Strategy
- 5/15 EMA crossover
- Regime-filtered entries using HMM
- Long trades only in Uptrend (+1)
- Short trades only in Downtrend (-1)
- No trades in Sideways (0)

### Backtesting Setup
- Train: First 70%
- Test: Last 30%
- Frequency: Daily

### Metrics Evaluated
- Total Return
- Sharpe Ratio
- Sortino Ratio
- Calmar Ratio
- Max Drawdown
- Win Rate
- Profit Factor
- Total Trades

### Conclusion
Integrating regime detection improves robustness by filtering
false signals in sideways markets and aligning trades with
dominant market conditions.




---

